# Chapter 10: Map scripting 

## 10.1 Introduction

This chapter describes the ArcPy mapping module, also referred to as arcpy.mapping.  
The ArcPy mapping module helps automate mapping tasks, including managing map documents, data frames, layer files, and the data within these files.  
There is also support for the automation of map export and printing, as well as the creation of PDF map books.  


## 10.2 Working with the ArcPy mapping module 

The ArcPy mapping module can be used to automate ArcMap workflows to speed up repetitive tasks. Some typical examples of uses for the ArcPy mapping module are as follows:  
* Finding a layer with a particular data source and replacing it with another data source  
* Modifying the display properties of a speci&c layer in multiple ArcMap documents  
* Generating reports that describe the properties of ArcMap documents, including data sources, layers with broken data links, information on the spatial reference of data frames, and more  

## 10.3 Opening map documents 

There are two ways to start working with a map document using the ArcPy mapping module:   
(1) use the map document from the current ArcMap session or  
(2) reference an existing .mxd &le stored on disk.  
The MapDocument function is used to accomplish both.  
The syntax of the MapDocument function is  

In [ ]:
MapDocument(mxd_path)

The path is a string representing an .mxd file on disk.  
The following code
opens a map document:

In [ ]:
mapdoc = arcpy.mapping.MapDocument(C:/Mapping/Study_Areas.mxd")

To use the current map document in ArcMap, the keyword CURRENT (in all uppercase letters) is used:

In [ ]:
mapdoc = arcpy.mapping.MapDocument("CURRENT")

When a MapDocument object is referenced in a script, the .mxd file is locked.  
This prevents other applications from making changes to the file.  
It is therefore good practice to remove the reference to a map document when it is no longer needed in a script by using the Python del statement.  
A mapping script therefore often has a structure that looks something like the following:  

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("C:/Mapping/Study_Areas.mxd")
# <code that modifies map document properties>
mapdoc.save()
del mapdoc

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("./Ch10/ch10.mxd")
# <code that modifies map document properties>
mapdoc.save()
del mapdoc

## 10.4 Accessing map document properties and methods 

In the following example, the CURRENT keyword is used to obtain the map document currently open in ArcMap, and the filePath property is used to print the system path for the .mxd file:

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("CURRENT")
path = mapdoc.filePath
print path
del mapdoc

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("./Ch10/ch10.mxd")
path = mapdoc.filePath
print path
del mapdoc

The following example updates the current map document's title and saves the .mxd file:   

In [ ]:
import arcpy  
mapdoc = arcpy.mapping.MapDocument("CURRENT")  
mapdoc.title = "Final map of study areas"  
mapdoc.save()  
del mapdoc  

As you review these basic examples, remember that they can be used to automate more complex tasks, such as making changes to multiple map documents rather than just the current one.


## 10.5 Working with data frames 

Map documents contain one or more data frames and each data frame typically contains one or more layers.  
Data frames and layers are perfect objects for use in lists, which can help automate tasks.  
The ListDataFrames function returns a list of DataFrame objects in a map document.  
The syntax is as follows  

In [ ]:
ListDataFrames(map_document, {wild_card})  

Once you have a list of data frames in a map document, you can look through them to examine or modify their properties. 
Running the following code prints a list of all the data frames in a map document:

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("CURRENT")
listdf = arcpy.mapping.ListDataFrames(mapdoc)
for df in listdf:
    print df.name
del mapdoc

If you want to work with just one of the data frames, you can use its index number, as follows:

In [ ]:
print listdf[0].name

The order of a list of data frames is the same as the order used in the Arcmap table of contents.  
  

DataFrame object properties, such as map extent, scale, rotation, and spatial reference, use map units.   
Other properties use page units to position and size the data frame on the layout page.   
Data frames are also used to access other objects - for example, the ListLayers function is used to access the layers in each dataframe.   
**You can then loop through the layers to get their properties.**   
It is therefore important to **uniquely** name the data frames within a single map document.  

There are quite a number of data frame properties, which are described in the ArcPy documentation in ArcGIS Desktop Help.  
The properties of the DataFrame object are a subset of all the properties on the Data Frame Properties dialog box in Arcmap (right-click a data frame in the Tab1e Of Contents window and click Properties).

Scripting does not provide access to all the properties on the Data Frame Properties dialog box, and conversely, some DataFrame object properties are not on the Data Frame Properties dialog box.  
For example, the scale of a data frame can be set using scripting, but in ArcMap, it is accomplished by using a tool on the Standard toolbar.

In most cases when you work with a map document, you are not interested in changing all the properties of a data frame , but only a few selected ones.  
For example, in the following code, the spatial reference of all data frames in a map document is set to the same spatial reference as that of a specific shapefile, and the scale of all data frames is set to 1:24,000:


In [ ]:
import arcpy
dataset = "C:/map/boundary.shp"
spatialRef = arcpy.Describe(dataset).spatalReference
mapdoc = arcpy.mapping.MapDocument("C:/map/final.mxd")
for df in arcpy.mapping.ListDataFrames(mapdoc):
    df.spatalReference = spatialRef
    df.scale = 24000
mapdoc.save()    
del mapdoc

In addition to the properties already discussed, the DataFrame object also has two methods: panToExtent and zoomToSelectedFeatures.  
The panToExtent method maintains the data frame scale but pans and cen ters the data frame extent based on the properties of an Extent object, which has to be provided as a parameter.  
An Extent object is a rectangle specified by providing the coordinates of the lower-left corner and the upper-right corner in map units.  
In most cases, this property is derived from an existing object, such as a feature or a layer.  
For example, the getExtent method can be used to obtain the extent of a layer.  
The follow ing code pans the extent of a data frame called df based on the extent of the features in a layer object called lyr:  


In [ ]:
df.panToExtent(lyr.getExtent())

The zoomToSelectedFeatures method is similar to the ArcMap opera tion Selection > Zoom to Selected Features.   
Running the following code zooms to the extent of all selected features in a data frame called df:  

In [ ]:
df.zoomToSelectedFeatures()

If no features are selected, the code will zoom to the full extent of alllayers.

## 10.6 Working with layers

A data frame typically contains one or more layers and the Layer object is essential to managing these layers.   
The Layer object provides access to many different layer properties and methods.   
There are two ways to reference Layer objects.   
The first approach is to use the Layer function to reference a layer (.lyr) file on disk.   
It is similar to how map document files (.mxd) are referenced.   
The syntax of the Layer function is  

In [ ]:
Layer(lyr_file_ path)

The parameter of the Layer function is the full path and file name of an
existing .lyr file. For example:      

In [ ]:
Lyr = arcpy.mapping.Laye("C:/Mapping/study.lyr")

The second approach is to use the ListLayers function to reference the layers in an .mxd file, or just the layers in a particular data frame in a map document, or the layers within a .lyr file.  
The syntax of the List La yers function is

In [ ]:
Layer(ly_rfile_path)

The parameter of the Layer function is the full path and file name of an
existing .lyr file. For example:

In [ ]:
Lyr = arcpy.maping.Layer("C:/Mappng/study.lyr")

The second approach is to use the ListLayers function to reference the layers in an .mxd file, or just the layers in a particular data frame in a map document, or the layers within a .lyr file.   
The syntax of the List La yers function is  

In [ ]:
ListLayers(map_document_or_layer, {wild_card}, {data_frame})

The only required element is a map document or layer file.  
An optional wild card can be used to limit the result.  
An optionaldata frame variable can be used that references a specific DataFrame object.  
For example, the follow ing code returns a list of all the layers in an ArcMap document, and then prints the names of all the layers:  

In [ ]:
import arcpy
myDoc = arcpy.mapping.MapDocument("CURRENT")
lyrlist = arcpy.mapping.ListLayers(mapdoc)
for lyr in lyrlist:
    print lyr.name

To access just the layers in a specific data frame, the Data Frame object has to be referenced as a parameter.   
In the following example, the st aye function returns only the layers in the data frame that have index number 0.   
The wild_card parameter is skipped using an empty string ("")  

In [ ]:
import arcpy
myDoc = arcpy.mapping.MapDocument("CURRENT")
dflist = arcpy.mapping.ListDataFrames(mapdoc)
lyrlist = arcpy.mapping.ListLayers(mapdoc, "", dflist[0])
for lyr in lyrlist:
    print lyr.name

The following code illustrates how to reference the layers in a .layer file on disk and print the name of layer objects:

In [ ]:
import arcpy
lyrfile = arcpy.mapping.Layer("C:/Data/mylayers.lyr")
lyrlist = arcpy.mapping.ListLayers(lyrfile)
for lyr in lyr list:
    print lyr.name

A few examples will serve to illustrate the use of layer properties.  
For example, the following code turns on all the labels for the layers in the current map document using the showLabels property:  

In [ ]:
import arcpy
myDoc = arcpy.mapping.MapDocument("CURRENT")
dflist = arcpy.mapping.ListDataFrames(mapdoc)
lyrlist = arcpy.mapping.ListLayers(mapdoc, "", dflist[0])
for lyr in lyrlist:
    lyr.showLabels = True
del lyrlist

Instead of changing the properties of all the layers in a map document or a data frame , the layer properties can also be used to find a layer with a particular name.  
For example, the following code searches for a layer called "hospitals":

In [ ]:
import arcpy
myDoc = arcpy.mapping.MapDocument("CURRENT")
lyrlist = arcpy.mapping.ListLayers(mapdoc)
for lyr in lyrlist :
    if lyr.name == "hospitals":
        lyr.showLabels = True
del lyrlist

Layer names can be a bit confusing.  
The name of a layer is w hat is shown in the ArcMap table of contents.  
This may or may not be the same as the name of the source dataset for the layer.  
In any case, the name of a lay does not have an extension.  
So the name of a feature class could be hospitals.shp, but as a layer in ArcMap, the name of the layer is hospitals.  


Also remember that strings are case sensitive, so Hospitals is different from hospitals.  
To make your statements insensitive to case, you can use basic string operators.   
For example:

In [ ]:
if lyr.name.lower() == "hospitals":

Because not all types of layers support the same properties, the supports method can be used to determine whether a layer supports a particular property.   
This makes it possible to test whether a layer supports a property before trying to get or set its value.   
This reduces the need for error checking.   
In the earlier example where the labels were shown for alllayers in a data frame, it would make sense to f1 rst use the supp orts method to test whether this property is supported for each layer.   
The syntax of the supports method is  

In [ ]:
supports(layer property)

The parameter, in this case, would consist of one of the Layer object properties, such as br ghtness contrast, datasetName, or others.  
The supports method returns a Boolean value, so the example code to test whether labeling is possible would look as follows:  

In [ ]:
import arcpy
myDoc = arcpy.mapping.MapDocument("CURRENT")
dflist = arcpy.mapping.ListDataFrames(mapdoc)
lyrlist = arcpy.mapping.ListLayers(mapdoc, "", dflist[0])
for lyr in lyrlist:
    if lyr.supports("SHOWLABELS") == True:
        lyr.showLabels = True
del lyrlist

If you are unsure whether a layer supports a particular property, use the supports method to test it.   
Otherwise, you will need to use an error trapping method, such as a try - except statement, which is covered in chapter 11.  
  
In addition to properties and methods of layer objects, there are several functions in the ArcPy mapping module that are specifically designed to manage layers within a data frame.   
These include the following:  

* **AddLayer** - makes it possible to add a layer to a data frame within a map document using general placement options.  
* **AddLayerToGroup** - makes it possible to add a layer to a group layer within a map document using general placement options.  
* **InsertLayer** - makes it possible to add a layer to a data frame or to a group layer within a map document.   
It provides a more precise way of positioning the layer by using a reference layer.  
* **MoveLayer** - makes it possible to move a layer to a specific location within a data frame or group layer within a map document.  
* **RemoveLayer** - makes it possible to remove a layer from a map document.  
* **UpdateLayer** - makes it possible to update the layer properties or just the symbology of a layer in a map document by extracting the information from a source layer.   

These functions all must reference an already existing layer.   
It can be a layer file on disk, a layer from within the same map document, or a layer from a different map document.   
Thus, these functions do not perform the task of adding data to a map document, as Add Data does in ArcMap.  

## 10.7 Fixing broken data sources

Broken data sources are very common, and fixing them manually can be tedio us.  
Scripting can be used to automate these corrections once the nature of the fix has been identified.  
Changes can be made to the map docu ment w ithout even opening it.  
Before examining th ese methods in more detail, a few definitions are in order:  

* **Worhspace** - a container for data.   
It can be a folder that contains shapefiles, a coverage, or a geodatabase - for example, mydata.

* **Worhspace path** - the system path to a workspace. It includes the drive letter where the folder is located and any subfolders - for example, C:\mydata.  
For a file-based geodatabase, it includes the name of the geodatabase - for example, C:\mydata\project.gdb.

* **Dαtaset** - the feature class or table in the workspace.   
It is the actual name on disk, not the name displayed in the ArcMap table of contents.
For a shapefile, it would be something like hospitals.shp. 
For a feature class in a geodatabase, it would be something like hospitals.

* **Dαta source** - the combination of workspace and dataset - for example, mydata\hospitals.shp or mydata\project.gdb\hospitals  

Prior to using a map document, you should check for broken data sources using the ListBrokenDataSources function.   
This arcpy.mapping function returns a Python list of layer objects within a map document or layer file that have broken connections to their original data source.  
The syntax is

In [ ]:
ListBrokenDataSources(map_document_or_layer)

The following code illustrates how this function can be used to print the names of the layers in a map document that have broken data sources:

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("CURRENT")
brokenlist = arcpy.mapping.ListBrokenDataSources(mapdoc)
for lyr in brokenlist:
    print lyr.name
del mapdoc

Running this code returns the nam es of the layers as they appear in the ArcMap table of contents.  
Instead of the name, the layer property dataSource can be used to see the current data source being referenced by the layer, as follows:

In [ ]:
print lyr.dataSource

Once it is established that data sources ne ed to be updated or fixed, the following methods can be applied to map documents, layers, or tables:  
  
* **```findAndReplaceWorkspacePaths```** and **```replaceWorkspaces```** - use to perform a find-and-replace operation on the workspace path and the workspace, respectively.   
This assumes that the datasets are correct.   
For example, you can change C:\mydata\hospitals.shp to C:\newdata\hospitals.shp, but the name of the dataset (in this case, hospitals.shp) cannot be modified.  
  
* **```replaceDataSource```** - use to perform a find-and-replace opera tion on the workspace and the dataset.   
You can modify both the workspace and the dataset, or just the dataset.   
For example, you can change C:\mydata\hospitals.shp to C:\mydata\newhospitals.shp.  

The following methods work on three different classes: MapDocument, Layer , and TableV ew objects.   
In total, there are six different methods:  

```
1. MapDocument.findAndReplaceWorkspacePaths
2. MapDocument.replaceWorkspaces
3. Layer.findAndReplaceWorkspacePath
4. Layer.replaceDataSource
5. TableView.findAndReplaceWorkspacePath
6. TableView.replaceDataSource
```

The syntax of Map Document.findAndReplaceWorkspacePaths is

In [ ]:
MapDocument.findAndReplaceWorkspacePaths(findworkspacepath, replace_workspace_path, {validate})

Running this code searches for and replaces the workspace paths of all layers and tables in a map document that share that workspace.   
For example, the following code replaces all the workspace paths in the current map document:  
  

In [ ]:
import arcpy  
mapdoc = arcpy.mapping.MapDocument("CURRENT")  
mapdoc.findAndReplaceWorkspacePaths("C:/mydata", "C:/newdata")  
mapdoc.save()  
del mapdoc  

The MapDocument.findAndReplaceWorkspacePaths method works on multiple workspace types at once, including shapefiles, file geodata bases, and others.   
However, the workspace type cannot be modified.   
The MapDocument.replaceWorkspaces method can be used to modify both the workspace path and the workspace type - for example, from a folder containing shapefiles to a file geodatabase.   
The m ethod works on only one workspace at a time but can be used multiple times if more than one workspace type needs to be replaced.   
The syntax of the MapDocument.replaceWorkspaces method is  

In [ ]:
MapDocument.replaceWorkspaces(old_workspace_path, old_workspace_type, new_workspace_path, new_workspace_type, {validate})

For example, in the following code, references to shapefiles are redirected to feature classes in a file geodatabase:

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("C:/mydata/project.mxd")
mapdoc.replaceWorkspaces("C:/mydata/shapes", "SHAPEFILEWORKSPACE", "C:/mydata/database.gdb", "FILEGDBWORKSPACE")
mapdoc.save()
del mapdoc

Notice exactly what happened here.   
The workspace is changed, but not the dataset.   
For example, if the data source for a layer was C:\mydata\hospitals.shp, it has been modified to C: \mydata\database.gdb\hospitals.   
Because the type of workspace is specified, the .shp extension for the datasets is auto matically removed.   
The example code assumes that feature classes with the exact same names as the shapefiles exist in the file geodatabase.   
Remember that paths are not case sensitive  

Valid workspace types are listed as follows:  
* ACCESS_WORKSPACE  
* ARCINFO_WORKSPACE  
* CAD_WORKSPACE  
* EXCEL_WORKSPACE  
* FILEGDB_WORKSPACE  
* OLEDB_WORKSPACE  
* PCCOVERAGE_WORKSPACE  
* RASTER_WORKSPACE  
* SDE_WORKSPACE  
* SHAPEFILE_WORKSPACE  
* TEXT_WORKSPACE  
* TIN_WORKSPACE  
* VPF_WORKSPACE  

Notice that "personal geodatabase" is not specifically included as a type instead ACCESS_WORKSPACE is used.

When workspaces in a map docurnent are modified, there are a few things that may not work:  
* Joins and relates associated with raster layers and stand-alone tables are not updated.  
* Definition queries may no longer work because a slightly different SQL syntax is used-for exarnple, between file geodatabases and personal geodatabases.   
A slight modificaion to the SQL staternent would fix this problem.  
* Label expressions rnay no longer work for the sarne reason.  

The methods discussed so far work on map documents.   
However, data sources can also be modified for individuallayers.  
The Layer.findAndReplaceWorkspacePath method works on a Layer object and performs a find-and-replace operation on the workspace path for a single layer in a map document or layer file.   
The syntax of this method is   

In [ ]:
Layer.findAndReplaceWorkspacePath(find_workspace_path, replace_workspace_path, {validate})

The following code modifies the workspace for a layer that references a particular feature class in a personal geodatabase.  
Only a portion of the full path of the data source is rep laced-in this case, using a different personal geodatabase:  

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("C:/mydata/project.mxd")
lyrlist = arcpy.mapping.ListLayers(mapdoc):
for lyr in lyrlist :
    if lyr.supports("DATASOORCE"):
        lyr.dataSource == "C:/mydata/database.gdb/hospitals":
            lyr.findAndReplaceWorkspacePath("database.gdb", "newdata.gdb")
mapdoc.save()
del mapdoc

The Layer.findAndReplaceWorkspacePath method assumes the dataset has not changed.  
The replaceDataSource method can be used to change both the workspace and the dataset.  
The syntax of this method is

In [ ]:
Layer.replaceDataSource(workspace_path, workspace type, dataset name, {validate})

The following code replaces a specific data source.   
In this case, the value of the dataSource property is used to determine whether a layer should have its data source updated:  

In [ ]:
import arcpy
mapdoc = arcpy.mapping.MapDocument("C:/mydata/project.mxd")
lyrlist = arcpy.mapping.ListLayers(mapdoc):
for lyr in lyrlist:
    if lyr.supports("DATASOURCE"):
        lyr.dataSource == "C:/mydata/hospitals.shp":
            lyr.replaceDataSource("C:/mydata/hospitals.shp", "SHAPEFILE_WORKSPACE", "C:/mydata/newhospitals.shp")
mapdoc.save()
del mapdoc

The findAndReplaceWorkspacePath and replaceDataSource methods also exist for TableView objects.   
The syntax for using these methods to work with single tables is very similar to the syntax for working with layers.  

## 10.8 Working with page layout elements 

## 10.9 Exporting maps 

## 10.10 Printing maps 

## 10.11 Working with PDFs 

## 10.12 Creating map books 

## 10.13 Using sample mapping scripts 

## Points to remember 